In [11]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [12]:
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'

CATEGORIES = ['MGMT_negative', 'MGMT_positive']

block_h, block_w = (150, 150)
# changed the stride
stride = 1

Case_Num = 5

In [13]:
# Function to create backup of the data and create working directory

def Backup():
    import shutil
    print('Backing up the files')
    # Copying data for backup
    try:
        shutil.copytree(PATH, Org_Dir)
    except:
        print('Backup already exists')
    # Copying data for working
    try:
        shutil.copytree(Org_Dir, Work_Dir)
    except:
        print('Working directory already exists')
    
    # Deleting folders
    for category in CATEGORIES:
        try:
            shutil.rmtree(PATH + category)
        except:
            print('Folder already deleted')
    print('Backup complete')

    

In [14]:
# Function Definition --> Generate Tumor Images of Patients
import os

def Generate_images():
    from tqdm import tqdm
    import cv2
    print('Generating Images')
    try:
        workdir = os.listdir(Work_Dir)
        if '.DS_Store' in workdir:
            workdir.remove('.DS_Store')
        for case in workdir:
            Case_Path = os.path.join(Work_Dir, case + '/')
            casepath = os.listdir(Case_Path)
            if '.DS_Store' in casepath:
                casepath.remove('.DS_Store')
            for split in casepath:
                Split_Path = os.path.join(Case_Path, split + '/')
                
                splitpath = os.listdir(Split_Path)
                if '.DS_Store' in splitpath:
                    splitpath.remove('.DS_Store')
                for category in tqdm(splitpath):
                    Category_Path = os.path.join(Split_Path, category + '/')
                    os.chdir(Category_Path)
                    list_of_patients = []
                    categorypath = os.listdir(Category_Path)
                    if '.DS_Store' in categorypath:
                        categorypath.remove('.DS_Store')
                    for file in tqdm(categorypath):
                        patient = file.split('_')[0] + '_' + file.split('_')[1]
                        if patient not in list_of_patients:
                            list_of_patients.append(patient)
                            flair = nib.load('{}_flair.nii.gz'.format(patient))
                            flair_data = flair.get_fdata()  # Converting nii to 3d np array
                            seg_mask = nib.load(
                                '{}_seg.nii.gz'.format(patient))
                            seg_mask_data = seg_mask.get_fdata()  # Converting nii to 3d np array
                            # print("Loaded flair and seg_mask for patient {}".format(patient))
                            #Extracting only those layers from mask hich have non zero values
                            z = np.any(seg_mask_data, axis=(0, 1))
                            # zmin & zmax saves the corresponding layer numbers of tumor regions
                            zmin, zmax = np.where(z)[0][[0, -1]]

                            #Creating a new mask to remove segmentation
                            d = seg_mask_data
                            for layer in range(zmin, zmax+1):
                                nonzero = np.nonzero(d[:, :, layer])
                                r = nonzero[0]
                                c = nonzero[1]
                                if (r.size == 0 or c.size == 0):
                                    continue
                                rmin = np.min(r)
                                rmax = np.max(r)
                                cmin = np.min(c)
                                cmax = np.max(c)
                                # Replacing tumor region values by 1
                                d[rmin:rmax+1, cmin:cmax+1, layer] = 1

                            #Multiplying flair data with new mask
                            tumor = np.multiply(flair_data, d)

                            #Removing zero valued layers
                            tumor_layers = tumor[:, :, ~
                                                 (tumor == 0).all((0, 1))]

                            #converting to png files
                            Cropped_list = []  # list containing cropped 2d layers of tumor region
                            for lay in range(0, tumor_layers.shape[2]):
                                coords = np.argwhere(tumor_layers[:, :, lay])
                                x_min, y_min = coords.min(axis=0)
                                x_max, y_max = coords.max(axis=0)
                                cropped = tumor_layers[x_min:x_max +
                                                       1, y_min:y_max+1, lay]
                                # normalization/scaling
                                cropped *= (255.0/cropped.max())
                                Cropped_list.append(cropped)

                            frame = 0
                            # print('Saving images')
                            for item in Cropped_list:
                                
                                # item is numpy.ndarray
                                im = item
                                # r = 150/im.shape[1]
                                dim = (block_h, block_w)
                                im_resized = cv2.resize(
                                    im,dim, interpolation=cv2.INTER_LANCZOS4)  
                                # print("read cv2")
                                cv2.imwrite("{}_img_{}.png".format(patient,frame),im_resized)
                                frame =frame+1
                                # im = Image.fromarray(item)
                                # im = im.convert('L')
                                # width, height = im.size
                                # print(item.shape[0], item.shape[1])
                                # print("width = {} height = {}".format(width, height))
                                # if ((item.shape[0]*item.shape[1]) >= 300):
                                #     print("here")
                                #     frame = frame + 1
                                #     im = cv2.imread(item)
                                #     im_resized = cv2.resize( im, fx=scale_x, fy=scale_y, interpolation=cv2.INTER_CUBIC)  # type: ignore
                                #     # im = im.convert('L')
                                #     # width, height = im.size
                                #     # if (height < 120 and width >= 120):
                                #     #     im = im.resize((150, width))
                                #     # elif (height >= 120 and width < 120):
                                #     #     im = im.resize((height, 32))
                                #     # elif (height < 120 and width < 120):
                                #     #     im = im.resize((120, 120))
                                #     im_resized.save("{}_img_{}.png".format(
                                #         patient, frame))
                                #     im_resized.close()
                                # frame=frame+1
                            

                            #Removing unwanted nii files
                            niipath = os.listdir(os.getcwd())
                            if '.DS_Store' in niipath:
                                niipath.remove('.DS_Store')
                            for item in niipath:
                                try:
                                    if item.startswith(patient) and item.endswith(".gz"):
                                        os.remove(item)
                                except Exception as e:
                                    print(e)
    except Exception as e:
        print(e)
    os.chdir(Work_Dir)
    print('Images Generated')

In [15]:
# Function Definition --> Dividing Datasets into number of cases for Cross Validation

def Creating_Cases(Case_Num):
    import shutil
    import os
    print('Creating Cases')
    for case in range(Case_Num):
        CASES.append('Case_' + str(case + 1))

    # Creating Case-1
    Case = Work_Dir + 'Case_1'
    try:
        shutil.copytree(Work_Dir, Case)
    except Exception as e:
        print(e)

    # Creating Remaining Cases
    try:
        for case in range(Case_Num - 1):
            shutil.copytree(Case, Work_Dir + 'Case_' + str(case+2))
    except Exception as e:
        print(e)

    # Deleting folder, listed in the CATEGORIES list, after creating Cases
    for cate in CATEGORIES:
        try:
            # Deleting Folders of CATEGORIES list
            shutil.rmtree(Work_Dir + cate)
        except Exception as e:
            print(e)
    print('Cases Created')


In [16]:
# Function Definition --> Separating_Test_Data to perform K-fold Cross Validation

def Separating_Test_Data():
    import splitfolders
    import shutil
    from tqdm import tqdm
    print('Seprating Test Data')
    i = 0
    workdir = os.listdir(Work_Dir)
    if '.DS_Store' in workdir:
        workdir.remove('.DS_Store')
        print("Removed .DS_Store")
    for case in tqdm(workdir):
        try:
            Case_Path = os.path.join(Work_Dir, case + '/')
            casepath = os.listdir(Case_Path)
            if '.DS_Store' in casepath:
                casepath.remove('.DS_Store')
            for Type in casepath:
                Type_Path = os.path.join(Case_Path, Type + '/')
                typepath = os.listdir(Type_Path)
                if '.DS_Store' in typepath:
                    typepath.remove('.DS_Store')
                for patient in typepath:
                    Patient_Path = os.path.join(Type_Path, patient + '/')
                    patientpath = os.listdir(Patient_Path)
                    if '.DS_Store' in patientpath:
                        patientpath.remove('.DS_Store')
                    for file in patientpath:
                        file_Path = os.path.join(Patient_Path, file)
                        source = file_Path
                        destination = Type_Path
                        shutil.copy(source, destination)
                    shutil.rmtree(Type_Path + patient)
        except Exception as e:
            print(e)
        input_folder = Case_Path
        output_folder = input_folder
        i = i+1
        splitfolders.ratio(input_folder, output=output_folder,
                           seed=i, ratio=(.8, .2), group_prefix=5)

        # Deleting folder, listed in the CATEGORIES list, after Seprating the data
        for cate in CATEGORIES:
            try:
                shutil.rmtree(Case_Path + cate)
            except Exception as e:
                print(e)

        # Rename folder 'val' to 'test'
        try:
            os.rename(Case_Path + 'val', Case_Path + 'test')
        except Exception as e:
            print(e)
    print('Test Data Seprated')


In [17]:
# Function Definition --> removing .DS_Store files to avoid errors in feautre labelling
def remove_ds_store():
    import os
    import shutil
    print('Removing .DS_Store')
    workdir = os.listdir(Work_Dir)
    if '.DS_Store' in workdir:
        workdir.remove('.DS_Store')
        print("Removed .DS_Store from Work_Dir : {}".format(Work_Dir))
    for case in workdir:
        try:
            Case_Path = os.path.join(Work_Dir, case + '/')
            casepath = os.listdir(Case_Path)
            if '.DS_Store' in casepath:
                casepath.remove('.DS_Store')
                print("Removed .DS_Store from Case_Path : {}".format(Case_Path))
            for Type in casepath:
                Type_Path = os.path.join(Case_Path, Type + '/')
                typepath = os.listdir(Type_Path)
                if '.DS_Store' in typepath:
                    typepath.remove('.DS_Store')
                    print("Removed .DS_Store from Type_Path : {}".format(Type_Path))
                for patient in typepath:
                    Patient_Path = os.path.join(Type_Path, patient + '/')
                    patientpath = os.listdir(Patient_Path)
                    if '.DS_Store' in patientpath:
                        patientpath.remove('.DS_Store')
                        print("Removed .DS_Store from Patient_Path : {}".format(Patient_Path))
                    for file in patientpath:
                        file_Path = os.path.join(Patient_Path, file)
                        if file_Path.endswith('.DS_Store'):
                            os.remove(file_Path)
                            print("Removed .DS_Store from file_Path : {}".format(file_Path))
        except Exception as e:
            print(e)


In [18]:
# All Function Calls
import os
import numpy as np
import nibabel as nib
from PIL import Image
import cv2

PATH = path + 'Data/BRATS/mod_data1/'
Org_Dir = PATH + 'Original_Data_Backup/'
Work_Dir = PATH + 'Working_Data/'

# Function Call --> Backup Original Data For Safety Purpose
# Backup()

# Function Call --> Dividing Datasets into number of cases for Cross Validation
CASES = []
Creating_Cases(Case_Num)

# Function Call --> Seprating_Test_Data to perform K-fold Cross Validation
Separating_Test_Data()

# Function Call --> Generate Tumor Images of Patients
Generate_images()

# Function Call --> Create Image Blocks
# Creating_Image_Blocks(block_h, block_w, stride)

# Function Call --> remove >.DS_store files to avoid errors in feautre labelling
remove_ds_store()

os.chdir(path)
print('All Done')
os.system('say "your program has finished"')


Creating Cases
Cases Created
Seprating Test Data
Removed .DS_Store


Copying files: 400 files [00:09, 44.19 files/s]
Copying files: 400 files [00:09, 42.46 files/s]
Copying files: 400 files [00:09, 43.81 files/s]
Copying files: 400 files [00:09, 43.59 files/s]
Copying files: 400 files [00:09, 44.37 files/s]
100%|██████████| 5/5 [01:29<00:00, 17.99s/it]


Test Data Seprated
Generating Images


100%|██████████| 2/2 [00:58<00:00, 29.02s/it]


Images Generated
Removing .DS_Store
Removed .DS_Store from Work_Dir : /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data1/Working_Data/
Removed .DS_Store from Case_Path : /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data1/Working_Data/Case_5/
Removed .DS_Store from Type_Path : /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data1/Working_Data/Case_5/test/
Removed .DS_Store from Type_Path : /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data1/Working_Data/Case_5/train/
Removed .DS_Store from Case_Path : /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data1/Working_Data/Case_2/
Removed .DS_Store from Case_Path : /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/Data/BRATS/mod_data1/Working_Data/Case_3/
Removed .DS_Store from Case_Path : /Users/vitthal/Documents/Research/DataScience/MedicalResearch/

0

In [19]:
# reconstucting working directory
def reconstruct():
    import shutil
    print('Reconstructing')
    # Deleting working directory
    shutil.rmtree(PATH+ "Working_data/")
    # Copying data from backup
    try:
        shutil.copytree(Org_Dir, Work_Dir)
    except:
        print('Working directory already exists')

    
    print('Reconstruction complete')


In [20]:
# reconstruct()